In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy.signal import find_peaks

In [5]:
# Initialize MetaTrader 5 connection
if not mt5.initialize():
    print("Initialization failed")
    quit()

NameError: name 'mt5' is not defined

In [9]:
#Download the OHLC data from the last 7 days
utc_from = datetime.utcnow() - timedelta(days=7)
utc_to = datetime.utcnow()

/var/folders/2q/hzgb6kgx0d7cswzg01w1q5qh0000gn/T/ipykernel_40193/3491965863.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  utc_from = datetime.utcnow() - timedelta(days=7)
/var/folders/2q/hzgb6kgx0d7cswzg01w1q5qh0000gn/T/ipykernel_40193/3491965863.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  utc_to = datetime.utcnow()
